In [1]:
pip install torch transformers sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.38.3 which is incompatible.



  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.4 MB 6.3 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.4 MB 6.6 MB/s eta 0:00:02
   --------------- ------------------------ 3.9/10.4 MB 6.5 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.4 MB 6.5 MB/s eta 0:00:01
   -------------------------- ------------- 6.8/10.4 MB 6.6 MB/s eta 0:00:01
   ------------------------------- -------- 8.1/10.4 MB 6.5 MB/s eta 0:00:01
   ----------------------------------- ---- 9.2/10.4 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------------------------------

In [3]:
# inference.py
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# 1. Load the model
def model_fn(model_dir):
    model = DistilBertForSequenceClassification.from_pretrained(model_dir)
    tokenizer = DistilBertTokenizer.from_pretrained(model_dir)
    return model, tokenizer

# 2. Parse the input
def input_fn(request_body, content_type):
    if content_type == 'application/json':
        import json
        input_data = json.loads(request_body)
        return input_data['text']  # Extract text from input
    else:
        raise ValueError(f"Unsupported content type: {content_type}")

# 3. Make the prediction
def predict_fn(input_data, model_and_tokenizer):
    model, tokenizer = model_and_tokenizer
    inputs = tokenizer(input_data, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return prediction

# 4. Format the output
def output_fn(prediction, accept):
    if accept == "application/json":
        import json
        return json.dumps({'sentiment': prediction}), accept
    else:
        raise ValueError(f"Unsupported accept type: {accept}")


In [5]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load pre-trained DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Save model and tokenizer locally
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

print("Model and tokenizer saved successfully!")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Model and tokenizer saved successfully!
